In [1]:
# modify these for your own computer
#repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

#data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'
data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import sys
import matplotlib.pyplot as plt



# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import mixture
from sklearn.feature_extraction.text import CountVectorizer


# our code
sys.path.append(repo_directory + 'code/')


sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
#from bag_of_words import load_tf_idf
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import re
import glob
import cPickle as pickle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords


from text_normalization import *
from pipeline_helper_functions import save_sparse_csr, load_sparse_csr

# compute bag of words matrix (bow matrix)

In [3]:
%%time

## compute bag_of_words matrix (copied from bag_of_words.py)

#tf_iter = textfile_iter(file_paths)
text_data = []
for fle in file_paths:
    text_data.append(open(fle, "r").read())
        
print len(text_data)
        


# stemmer and stop words
stemmer = PorterStemmer()
stop_words = stopwords.words("english")
min_df = 1  # be careful about integer/float, see sklean documentation
max_df = 1.0

# stem stop words
stop_words = [stemmer.stem(w) for w in stop_words]

bag_of_words = CountVectorizer(min_df=min_df,
                               max_df=max_df,
                               tokenizer=StemTokenizer(stemmer),
                               stop_words=stop_words)

#bow_matrix = bag_of_words.fit_transform(file_paths)
bow_matrix = bag_of_words.fit_transform(text_data)

27885
Wall time: 1h 40s


In [5]:
bow_matrix

<27885x567570 sparse matrix of type '<type 'numpy.int64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

# store bow matrix information (vocab, matrix, row indices) in new directory: 'nlp_bow_dir'

In [14]:
if not os.path.exists(nlp_bow_dir):
    os.mkdir(nlp_bow_dir)

In [15]:
# vocabulary
vocab = bag_of_words.get_feature_names()

# map from CL opinion ids to bow matix index
op_id_to_bow_id = {re.search(r'(\d+)\.txt', file_paths[i]).group(1): i for
                   i in range(len(file_paths))}

# save data
save_sparse_csr(nlp_bow_dir + 'bag_of_words_matrix', bow_matrix)

with open(nlp_bow_dir + 'op_id_to_bow_id.p', 'wb') as fp:
    pickle.dump(op_id_to_bow_id, fp)

with open(nlp_bow_dir + 'vocab.p', 'wb') as fp:
    pickle.dump(vocab, fp)

# transform the bow matrix to tf-idf matrix using 'TfidfTransformer'

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

In [7]:
%%time
# compute tfidf matrix from bag of words

tfidf = TfidfTransformer()

tfidf_matrix = tfidf.fit_transform(bow_matrix)

Wall time: 3.3 s


In [8]:
tfidf_matrix

<27885x567570 sparse matrix of type '<type 'numpy.float64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

# store tfidf matrix information (vocab, matrix, row indices) in new directory: 'nlp_sub_dir'

In [9]:
if not os.path.exists(nlp_sub_dir):
    os.mkdir(nlp_sub_dir)

In [10]:
# vocabulary
vocab = tfidf.get_feature_names()

# map from CL opinion ids to bow matix index
op_id_to_bow_id = {re.search(r'(\d+)\.txt', file_paths[i]).group(1): i for
                   i in range(len(file_paths))}



# save data
save_sparse_csr(output_dir + 'tfidf_matrix', tfidf_matrix)

# saved row indices
with open(output_dir + 'op_id_to_bow_id.p', 'wb') as fp:
    pickle.dump(op_id_to_bow_id, fp)

# save vocab
with open(output_dir + 'vocab.p', 'wb') as fp:
    pickle.dump(vocab, fp)

AttributeError: 'TfidfTransformer' object has no attribute 'get_feature_names'

# is it ok to save the 'vocab' from 'bag_of_words.get_feature_names()'?

In [12]:
# vocabulary
vocab = bag_of_words.get_feature_names()

# map from CL opinion ids to bow matix index
op_id_to_bow_id = {re.search(r'(\d+)\.txt', file_paths[i]).group(1): i for
                   i in range(len(file_paths))}



# save data
save_sparse_csr(nlp_sub_dir + 'tfidf_matrix', tfidf_matrix)

# saved row indices
with open(nlp_sub_dir + 'op_id_to_bow_id.p', 'wb') as fp:
    pickle.dump(op_id_to_bow_id, fp)

# save vocab
with open(nlp_sub_dir + 'vocab.p', 'wb') as fp:
    pickle.dump(vocab, fp)

In [13]:
vocab

[u'0',
 u'00',
 u'000',
 u'0000',
 u'00000',
 u'000000',
 u'0000000',
 u'000006',
 u'000007',
 u'00002',
 u'000027775102000049',
 u'000027775102000049txt',
 u'00003',
 u'000033',
 u'000033\u201d',
 u'00004',
 u'00005',
 u'00006',
 u'00007',
 u'00008',
 u'00009',
 u'0001',
 u'00010',
 u'00011',
 u'000119312503085265d424b5htm',
 u'00013',
 u'00014',
 u'00015',
 u'00017',
 u'00019',
 u'000190',
 u'00021',
 u'00021bi',
 u'00023',
 u'00024',
 u'00025',
 u'00026',
 u'00027',
 u'0003',
 u'00031',
 u'000311\u201312',
 u'00032',
 u'0003232',
 u'00033',
 u'00034',
 u'00035',
 u'00038',
 u'000390',
 u'0003\u2013hlm',
 u'0004',
 u'00041',
 u'000418',
 u'00042',
 u'00044',
 u'0004478798',
 u'00046',
 u'00048',
 u'00049',
 u'00050',
 u'00052',
 u'00055',
 u'00056',
 u'00057',
 u'00058',
 u'00059',
 u'0006',
 u'00061',
 u'00062',
 u'00063',
 u'00064',
 u'00065',
 u'00066',
 u'00067',
 u'00069',
 u'00070',
 u'00071',
 u'00072',
 u'00073',
 u'00075',
 u'00076',
 u'00077',
 u'00079',
 u'00088',
 u'00090

# Clustering Work

In [7]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# largest connected component

restrict our attention to the largest connected componenet on the network. also we are missing some text files from 2016 so lets ignore 2016.

In [8]:
# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

# graph clustering

Do community detection on network

## modularity on undirected scotus

In [9]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

print mod_clust.summary()

Clustering with 24724 elements and 126 clusters
Wall time: 1min 33s


In [123]:
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

## walktrap on undirected scotus

In [11]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 28s


In [127]:
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

# NLP Clustering

## load tf-idf vectors, bow vectors

In [25]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

In [26]:
def load_bow(nlp_dir):
    """
    bow_matrix, op_id_to_bow_id = load_bow(nlp_dir)
    """
    bow_matrix = load_sparse_csr(nlp_dir + 'bag_of_words_matrix.npz')

    with open(nlp_dir + 'op_id_to_bow_id.p', 'rb') as f:
        op_id_to_bow_id = pickle.load(f)

    with open(nlp_dir + 'vocab.p', 'rb') as f:
        vocab = pickle.load(f)

    return bow_matrix, op_id_to_bow_id, vocab

In [27]:
bow_matrix, op_id_to_bow_id_2, vocab_2 = load_bow(nlp_bow_dir)

## K means clustering on tf-idf

In [90]:
%%time

# set number of clusters
num_clusters = 30

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_tfidf_clusters = km.labels_.tolist()

Wall time: 1h 10min 5s


## Gaussian Mixture Model Clustering

In [146]:
%%time

# set number of clusters

# run GMM
gmm = mixture.GaussianMixture(n_components=1)
gmm.fit(tfidf_matrix)

gmm_clusters = gmm.labels_.tolist()

AttributeError: 'module' object has no attribute 'GaussianMixture'

## Compare NLP clustering (tfidf) vs graph clustering

In [128]:
#clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])
clusters = pd.DataFrame(index=op_id_to_bow_id, columns=['km', 'mod'])

# add in communities 
clusters['mod'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['mod'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['mod'] = clusters['mod'].astype(np.int)

# add in NLP clusters
clusters['km'] = nlp_tfidf_clusters

# add in walktrap clusters
clusters['wt'] = walktrap_clusters
clusters['wt'].fillna(max(walktrap_clusters) + 1, inplace=True)
clusters['wt'] = clusters['wt'].astype(np.int)

clusters.to_csv("clusters.csv")

In [145]:
clusters

,km,mod,wt
145658,18,1,5
89370,26,3,294
89371,13,0,35
89372,17,0,3
89373,5,0,3
89374,29,2,4
89375,20,2,5
89376,10,2,6
89377,15,2,7
89378,12,2,7


# summarize cluster function

In [28]:
def summarize_cluster(opinions, num_words, tfidf_matrix, op_id_to_bow_id, vocab):
    """
    This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

    Parameters
    -----------
    opinions: list of opinion ids
    num_words: number of words to return as the summary
    tfidf_matrix: the tf-idf matrix of all SCOTUS opinions
    op_id_to_bow_id: dict that maps opinion ids to rows of the tfidf matrix

    Output
    -------
    a list of the words with highest tf-idf scores amount the given opinions
    """
    
    
    # op_id_to_bow_id: opinion_id = row_index
    
    vocab = np.array(vocab)
    n = num_words
    top_list = []
    for each_opinion in opinions:
        row_index = op_id_to_bow_id[each_opinion]
                
        # get the sorted column indices (by tf-idf value) for the respective row_index/opinion_file
        column_ind = np.argsort(tfidf_matrix[row_index].toarray()).flatten()[::-1]
        
        print column_ind
        
        # top vocabs for each document
        top_words = vocab[column_ind][:n]
        
        # store all the vocabs into a list
        top_list.append(top_words)
        
    return top_list

In [29]:
opinions = ['1722', '1723']
summarize_cluster(opinions, 2, tfidf_matrix, op_id_to_bow_id, vocab)

[547175 453648 286958 ..., 377712 377713      0]
[254606 323880 403124 ..., 377945 377946      0]


[array([u'\xa7901b', u'rule'], 
       dtype='<U92'), array([u'deport', u'immigr'], 
       dtype='<U92')]

# Get List of All Text Files (opinions basically)

In [104]:
text_file_nums = []
for i in file_paths:
    num = re.search(r'(\d+)', i)
    num = num.group()
    text_file_nums.append(num)

implement a few more summarize_cluster functions  

compute the mean tf-idf vector of the cluster, return the top K words from this mean vector  
compute the mean tf-idf vector, return the document in the cluster closet to the mean  
compute the mean tf-idf vector of the cluster and also of the complement of the cluster, take the difference mu_cluster -   mu_complement, return the top K words in this difference  
cluster the documents using a gaussian mixture model (e.g. http://scikit-learn.org/stable/modules/mixture.html)  

In [6]:
nlp_tfidf_clusters

NameError: name 'nlp_tfidf_clusters' is not defined

In [163]:
labels_.tolist()

AttributeError: 'KMeans' object has no attribute 'labels_'

In [158]:
cc = km.cluster_centers_

AttributeError: 'KMeans' object has no attribute 'cluster_centers_'

In [ ]:
'''
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector  



Like the mean vector over all vectors in the cluster

[11:55]  
Eg if there are two vectors in the cluster then you add their tf-idf vectors and divide by 2
'''

In [21]:
sentence_list=['hello how are you', "I am doing great", "my name is abc"]

vectorizer=TfidfVectorizer(min_df=1, max_df=0.9, stop_words='english', decode_error='ignore')
vectorized=vectorizer.fit_transform(sentence_list)

km=KMeans(n_clusters=3, init='k-means++',n_init=10, verbose=1).fit(vectorized)
print km

Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0: center shift 0.000000e+00 within tolerance 1.666667e-05
Initialization complete
Iteration  0, inertia 0.000
Converged 

In [22]:
print km
print km.labels_

poop = km.labels_.tolist() # when calling .tolist(), you ahve to assign this to a variable, otherwise you get an error
print poop

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)
[1 2 0]
[1, 2, 0]
[1, 2, 0]


In [13]:
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)